In [324]:
import pandas as pd
import pandasql as ps
import numpy as np
import time
import re

## Load Dataframe

In [325]:
# Open dataframe

one_piece_rt_df = pd.read_csv("one_piece_RT.csv")

In [326]:
print(len(one_piece_rt_df))

1230


In [327]:
one_piece_rt_df.head(5)

,Unnamed: 0,User_IDs,Timestamps,Reviews,Source
0,0,Deen W,Dec 20,One of the best manga adaptations of all time....,Rotten Tomatoes
1,1,Gol D,Dec 6,Best Anime Live Action Adaptation of All Time....,Rotten Tomatoes
2,2,Celeborn G,Nov 30,How were they able to cram so much cringe into...,Rotten Tomatoes
3,3,Robert M,Oct 27,One Piece is an incredibly well done show. It ...,Rotten Tomatoes
4,4,Tygari D,Sep 30,i am not a one piece fan. but this live action...,Rotten Tomatoes


## Data Cleaning

### I. Checking Data

In [328]:
# Check duplicated values

print(one_piece_rt_df['Reviews'].duplicated().sum())

20


In [ ]:
# There are 20 duplicated reviews

duplicated_rows = one_piece_rt_df[one_piece_rt_df['Reviews'].duplicated(keep=False)]

duplicated_rows

,Unnamed: 0,User_IDs,Timestamps,Reviews,Source
1150,1150,Austin W,09/01/2023,Honestly? I couldn't have asked for a better a...,Rotten Tomatoes
1151,1151,Mack S,09/01/2023,Actually the best live action anime adaptation...,Rotten Tomatoes
1152,1152,A L,09/01/2023,Excellent adaptation with tweaks made that dev...,Rotten Tomatoes
1153,1153,John R,09/01/2023,"There were some corny, clunky bits here and th...",Rotten Tomatoes
1154,1154,Faiber,09/01/2023,No se esperaba mucho de un live action. Pero l...,Rotten Tomatoes
1155,1155,C J,09/01/2023,Awesome anime adaptation and in-general just a...,Rotten Tomatoes
1156,1156,Edox25,09/01/2023,This series is amazing,Rotten Tomatoes
1157,1157,FENG C,09/01/2023,"Excellent casting, embarrassing acting, very d...",Rotten Tomatoes
1158,1158,Arga L,09/01/2023,"it's just good,but it's can be better some thi...",Rotten Tomatoes
1159,1159,Maxim-Alexandre L,09/01/2023,It's been a cliché for years now that an anime...,Rotten Tomatoes


In [330]:
# There are 20 duplicated rows in database
# Need to drop them

one_piece_rt_df = one_piece_rt_df.drop_duplicates(['Reviews'], keep = 'first')

In [331]:
# Check if duplicated values are droped

len(one_piece_rt_df)

1210

In [332]:
# Before cleaning data, make sure there's no empty values

def missing_values(df):
    columns = df.columns
    for c in columns:
        missing = df[c].isnull().sum()
        print(f"The column, {c}, has {missing} values.")

In [333]:
missing_values(one_piece_rt_df)

The column, Unnamed: 0, has 0 values.
The column, User_IDs, has 2 values.
The column, Timestamps, has 0 values.
The column, Reviews, has 0 values.
The column, Source, has 0 values.


In [334]:
# We have 2 missing values on User IDs.
# However, since this is about sentiment analysis, decided to get rid of the User_IDs column as 1st cleaning

one_piece_rt_df = one_piece_rt_df.drop(["Unnamed: 0", "User_IDs"], axis=1)

In [335]:
one_piece_rt_df.head(5)

,Timestamps,Reviews,Source
0,Dec 20,One of the best manga adaptations of all time....,Rotten Tomatoes
1,Dec 6,Best Anime Live Action Adaptation of All Time....,Rotten Tomatoes
2,Nov 30,How were they able to cram so much cringe into...,Rotten Tomatoes
3,Oct 27,One Piece is an incredibly well done show. It ...,Rotten Tomatoes
4,Sep 30,i am not a one piece fan. but this live action...,Rotten Tomatoes


### II. Change Timestamps to Proper Format

In [336]:
# As some of them are partially recorded, they need to be changed to proper format

def time_conversion(text):
    if pd.isna(text):
        return pd.NaT
    
    now = pd.Timestamp.now()

    day_match = re.search(r'(\d+)\s*d', text)
    if day_match:
        day = int(day_match.group(1))
        return pd.Timestamp.now() - pd.to_timedelta(day, unit='d')
    
    partial_date = re.match(r'^[A-Za-z]{3}\s+\d{1,2}$', text)
    if partial_date:
        return pd.to_datetime(f"{text} {now.year}", format="%b %d %Y", errors="coerce")
    
    else:
        return pd.to_datetime(text, errors="coerce")
    

In [337]:
one_piece_rt_df['review_dates'] = one_piece_rt_df['Timestamps'].apply(time_conversion)

one_piece_rt_df['review_dates'] = one_piece_rt_df['review_dates'].dt.strftime('%Y-%m-%d')

In [338]:
# Once review_dates are all changed, drop Timestamps

one_piece_rt_df = one_piece_rt_df.drop("Timestamps", axis=1)

### III. VADER for Checking Sentiment

In [339]:
# Now it's time to check the sentiment using VADER

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment_vader = SentimentIntensityAnalyzer()

one_piece_rt_df['vader_score_review'] = one_piece_rt_df['Reviews'].apply(lambda text: sentiment_vader.polarity_scores(text)['compound'])

In [340]:
def sentiment_category_vader(sentiment):
        if sentiment >= 0.05:
            return "Positive"
        elif sentiment <= -0.05:
            return "Negative"
        else:
            return "Neutral"
        
one_piece_rt_df['vader_sentiment_review'] = one_piece_rt_df['vader_score_review'].apply(sentiment_category_vader)

In [341]:
one_piece_rt_df.head(3)

,Reviews,Source,review_dates,vader_score_review,vader_sentiment_review
0,One of the best manga adaptations of all time....,Rotten Tomatoes,2026-12-20,0.8402,Positive
1,Best Anime Live Action Adaptation of All Time....,Rotten Tomatoes,2026-12-06,0.6696,Positive
2,How were they able to cram so much cringe into...,Rotten Tomatoes,2026-11-30,0.4601,Positive


### IV. Last Step for Data Cleaning

In [342]:
# We all have necessary data, so let's organize dataframe by using SQL

one_piece_rt_df_final = ps.sqldf("select [review_dates], [Reviews], [vader_score_review], [vader_sentiment_review], [Source] from one_piece_rt_df")

In [343]:
# Finally, standardize column names by changing them to small letters

one_piece_rt_df_final = one_piece_rt_df_final.rename(columns = {'Reviews':'reviews', 'Source':'source'})

## Finalizing Data Cleaning Process & End

In [344]:
one_piece_rt_df_final.to_csv('one_piece_rt_df_cleaned.csv', index=False)